# Example 03: Get available dataflows from ILO API

This examples shows all the dataflows that are available from ILO's SDMX API (ILOSTAT SDMX Gateway). 

### Load required libraries

In [14]:
import xmltodict
import json
import requests 
import xml.etree.ElementTree as ET 

### Interactive shell

In [15]:
# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get data from API endpoint and save as xml file on disk

In [52]:
baseURL = 'http://www.ilo.org/ilostat/sdmx/ws/rest/'

# url of API endpoint returning all the dissemination dataflows (i.e. indicators) with data available
url = baseURL + 'dataflow/ILO/'  # 

# creating HTTP response object from given url 
resp = requests.get(url) 

# saving the xml message into an xml file 
with open('test.xml', 'wb') as f: 
    f.write(resp.content) 

347289

### Open xml file using `xmltodict.parse`
This will create an object consisting of various nested ordered dictionaries. Every node of the XML tree is an ordered dictionary of key-value pairs. The root of the XML tree is an ordered dictionary itself.

In [53]:
xmlFile = 'test.xml'

with open(xmlFile, encoding="utf-8") as fd:
    doc = xmltodict.parse(fd.read())


### Drill down to level 3, which contains the actual code list

In [55]:

print('---Root:')
len(doc)
rootKey = list(doc.keys())[0]
rootKey

print('---Level 1:')
level1 = doc[rootKey]
len(level1)
level1Keys = list(level1.keys())
level1Keys

print('---Level 2 / mes:Dataflows')
level2 = level1['mes:Dataflows']
len(level2)
level2Keys = list(level2.keys())
level2Keys

print('---Level 3 / str:Dataflow')
level3 = level2['str:Dataflow']
len(level3)

level3[0].keys()


---Root:


1

'mes:Structure'

---Level 1:


4

...

1

['str:Dataflow']

---Level 3 / str:Dataflow


400

odict_keys(['@agencyID', '@id', '@urn', '@version', '@isFinal', 'str:Name', 'str:Description', 'str:KeyFamilyRef'])

### Collect the concept scheme information in a dictionary

In [56]:
dataFlows = []

for df in level3:
    dataFlow = {}

    dataFlow['agencyId'] = df['@agencyID']
    dataFlow['id'] = df['@id']
    dataFlow['urn'] = df['@urn']
    dataFlow['version'] = df['@version']
    dataFlow['isFinal'] = df['@isFinal']
    
    dataFlow['nameEN'] = None
    dataFlow['nameES'] = None
    dataFlow['nameFR'] = None

    dataFlowName = df['str:Name']

    if isinstance(dataFlowName, list):
        for x in dataFlowName:
            if x['@xml:lang'] == 'en':
                dataFlow['nameEN'] = x['#text']
            if x['@xml:lang'] == 'es':
                dataFlow['nameES'] = x['#text']
            if x['@xml:lang'] == 'fr':
                dataFlow['nameFR'] = x['#text']
    else:
        if dataFlowName['@xml:lang'] == 'en':
            dataFlow['nameEN'] = dataFlowName['#text']
        if dataFlowName['@xml:lang'] == 'es':
            dataFlow['nameES'] = dataFlowName['#text']
        if dataFlowName['@xml:lang'] == 'fr':
            dataFlow['nameFR'] = dataFlowName['#text']
            
    
    
    dataFlow['descriptionEN'] = None
    dataFlow['descriptionES'] = None
    dataFlow['descriptionFR'] = None
            
    if 'str:Description' in df.keys():
        dataFlowDescription = df['str:Description']

        if isinstance(dataFlowDescription, list):
            for x in dataFlowDescription:
                if x['@xml:lang'] == 'en':
                    dataFlow['descriptionEN'] = x['#text']
                if x['@xml:lang'] == 'es':
                    dataFlow['descriptionES'] = x['#text']
                if x['@xml:lang'] == 'fr':
                    dataFlow['descriptionFR'] = x['#text']
        else:
            if dataFlowDescription['@xml:lang'] == 'en':
                dataFlow['descriptionEN'] = dataFlowDescription['#text']
            if dataFlowDescription['@xml:lang'] == 'es':
                dataFlow['descriptionES'] = dataFlowDescription['#text']
            if dataFlowDescription['@xml:lang'] == 'fr':
                dataFlow['descriptionFR'] = dataFlowDescription['#text']
   
    
    keyFamilyRef = df['str:KeyFamilyRef'] 
    
    dataFlow['keyFamilyID'] = keyFamilyRef['str:KeyFamilyID']
    dataFlow['keyFamilyAgencyID'] = keyFamilyRef['str:KeyFamilyAgencyID']
    
    
    dataFlows.append(dataFlow)
    
dataFlows
    

[{'agencyId': 'ILO',
  'id': 'DF_CP_ALL_ALL',
  'urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=ILO:DF_CP_ALL_ALL(1.0)',
  'version': '1.0',
  'isFinal': 'true',
  'nameEN': 'All Indicators',
  'nameES': None,
  'nameFR': None,
  'descriptionEN': 'All Indicators',
  'descriptionES': None,
  'descriptionFR': None,
  'keyFamilyID': 'CP_ALL_ALL',
  'keyFamilyAgencyID': 'ILO'},
 {'agencyId': 'ILO',
  'id': 'DF_CP_ALL_EAP_DWAF_NOC_RT',
  'urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=ILO:DF_CP_ALL_EAP_DWAF_NOC_RT(1.0)',
  'version': '1.0',
  'isFinal': 'true',
  'nameEN': 'Labour force participation rate, women',
  'nameES': 'Tasa de participación en la fuerza de trabajo, mujeres',
  'nameFR': "Taux de participation de la main-d'oeuvre, femmes",
  'descriptionEN': None,
  'descriptionES': None,
  'descriptionFR': None,
  'keyFamilyID': 'CP_ALL_EAP_DWAF_NOC_RT',
  'keyFamilyAgencyID': 'ILO'},
 {'agencyId': 'ILO',
  'id': 'DF_CP_ALL_EAP_DWAM_NOC_RT',
  'urn': 'urn:sdmx: